In [1]:
import pandas as pd
from os import listdir
from tqdm import tqdm

In [4]:
path = 'C:\\Users\\taest\\.zipline\\random_stocks' ## 데이터 위치

In [2]:
import pandas as pd
from io import StringIO

# Your data in a string format
data = """
trade_date,open,high,low,close,volume,dividend,in_sp500
1999-11-18,50.243098,50.243098,50.243098,50.243098,58425783.68,0.0,0
1999-11-19,51.092207,51.092207,51.092207,51.092207,19084157.70,0.0,0
"""

# Read the data into a DataFrame
df = pd.read_csv(StringIO(data), parse_dates=[0], index_col=[0])
df

,open,high,low,close,volume,dividend,in_sp500
trade_date,,,,,,,
1999-11-18,50.243098,50.243098,50.243098,50.243098,58425783.68,0.0,0
1999-11-19,51.092207,51.092207,51.092207,51.092207,19084157.70,0.0,0


In [20]:
def random_stock_data(environ,
                      asset_db_writer,
                      minute_bar_writer,
                      daily_bar_writer,
                      adjustment_writer,
                      calendar,
                      start_session,
                      end_session,
                      cache,
                      show_progress,
                      output_dir):
    """
    bundle insgest를 하려면 위의 인자들이 모두 전달 되어야 한다.
    divs / splits / metadata가 정의되고
    sessions / daily_bar_writer / assest_db_writer / adjustment_writer 가 작성된다.
    sessions은 데이터의 기간 정보를 가지고 있다.
    데이터를 지정된 폴더에 symbol 이름으로 csv 파일로 구성되어 있다.

    """
    # 경로로부터 파일 리스트를 얻는다.
    # 마지막 부분을 슬라이싱한다.
    symbols = [f[:-4] for f in listdir(path)]

    if not symbols:
        raise ValueError("No symbols found in folder.")

    # 배당금을 위한 빈 DataFrame을 준비한다.
    divs = pd.DataFrame(columns=['sid',
                                 'amount',
                                 'ex_date',
                                 'record_date',
                                 'declared_date',
                                 'pay_date']
                        )
    # 주식 분할을 위한 빈 DataFrame을 준비한다.
    splits = pd.DataFrame(columns=['sid',
                                   'ratio',
                                   'effective_date']
                          )
    # 메타데이터를 위한 빈 DataFrame을 준비한다.
    metadata = pd.DataFrame(columns=['start_date',
                                     'end_date',
                                     'auto_close_date',
                                     'symbol',
                                     'exchange'
                                     ]
                            )

    sessions = calendar.sessions_in_range(start_session, end_session)
    # 모든 주식에 대한 데이터를 얻고 Zipline에 쓴다. // 가격 데이터
    daily_bar_writer.write(process_stocks(symbols, sessions, metadata, divs))
    # 메타데이터를 쓴다.
    asset_db_writer.write(equities=metadata)
    # 주식 분할과 배당금을 쓴다.
    adjustment_writer.write(splits=splits,
                            dividends=divs)


# 주식을 반복 시행하기 위한 Generator 함수
# 역사적 데이터, 메타데이터와 배당 데이터를 구축한다.
def process_stocks(symbols:list, sessions, metadata:pd.DataFrame, divs:pd.DataFrame):
    for sid, symbol in tqdm(enumerate(symbols)):
        # print(f"Loading {symbol}...")

        df = pd.read_csv(f"{path}/{symbol}.csv", index_col=[0], parse_dates=[0])

        # 처음과 마지막 날짜 체크
        start_date = df.index[0]
        end_date = df.index[-1]

        # 공식 거래소 캘린더와 동기화
        df = df.reindex(sessions.tz_localize(None))[start_date:end_date]

        # 결측치 전방 채우기
        df.ffill(inplace=True)

        # 남은 NaN를 삭제한다.
        df.dropna(inplace=True)

        # auto_close 날짜는 마지막 거래 후의 날짜다.
        ac_date = end_date + pd.Timedelta(days=1)
        # 메타데이터 DataFrame에 행을 더한다.
        metadata.loc[sid] = start_date, end_date, ac_date, symbol, 'NYSE'

    # 만약 배당 데이터가 있다면, 이를 배당 DataFrame에 더한다.
    if 'dividend' in df.columns:
        # 배당이 있는 날짜들을 슬라이싱한다.
        tmp = df[df['dividend'] != 0.0]['dividend']
        div = pd.DataFrame(data=tmp.index.tolist(), columns=['ex_date'])

        # 지금 이 데이터를 가지고 있지 않으므로 빈 열을 제공한다.
        div['record_date'] = pd.NaT
        div['declared_date'] = pd.NaT
        div['pay_date'] = pd.NaT

        # 배당을 저장하고 증권 ID를 설정한다.
        div['amount'] = tmp.tolist()
        div['sid'] = sid

        # 마지막에 남긴 곳에서 숫자 매기기를 시작한다.
        ind = pd.Index(range(divs.shape[0], divs.shape[0] + div.shape[0]))
        div.set_index(ind, inplace=True)

        # 이 주식 배당을 모든 배당 리스트에 결합(append)한다.
        divs = pd.concat([divs.dropna(), div.dropna()], ignore_index=True)

    yield sid, df